In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

# Create an instance of TavilySearchResults with k=6 for retrieving up to 6 search results
search_tool = TavilySearchResults(k=10, max_results=10)

In [3]:
# 템플릿 수정: search_results를 제거하거나 처리 방법 변경
TEMPLATE = """당신은 맛집 블로거 입니다.

search_tool을 사용하여 검색 후 모든 결과를 분석하여 정확하게 답변해주세요.

확인된 주소는 get_coordinates 도구를 사용하여 좌표를 추출해주세요.

사용자 질문:
{input}

출력 형식:

{{
    "answer": "아주 간단한 답변 내용",
    "infos": [
        {{
            "name": "식당 이름",
            "address": "식당 주소",
            "subway": "식당 지하철역",
            "lat": "식당 위도",
            "lng": "식당 경도",
            "menu": "메뉴1, 메뉴2, ...",
            "review": "식당 후기",
            "url": ["url1", "url2", "url3", ...]
        }}
    ]
}}
{agent_scratchpad}
"""

from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
import requests
import os

prompt = ChatPromptTemplate.from_template(template=TEMPLATE)

# LLM 생성
llm = ChatOpenAI(model="gpt-4o", temperature=0)


# search_tool을 LangChain 형식에 맞게 변환
@tool
def search(query: str) -> list:
    """Search for information about restaurants and food."""
    return search_tool.invoke(query)


@tool
def get_coordinates(address: str) -> list:
    """Get the coordinates of a restaurant."""
    coordinate_url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {os.getenv('KAKAO_API_KEY')}"}
    params = {"query": address}
    response = requests.get(coordinate_url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
    if data["documents"]:
        location = data["documents"][0]
        latitude = location["y"]
        longitude = location["x"]

        return latitude, longitude


# 도구 리스트 생성
tools = [search, get_coordinates]

# 에이전트 생성
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# 에이전트 실행 예제
result = agent_executor.invoke({"input": "논현역 양식 맛집에 대해서 많이 찾아줘"})
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `search` with `{'query': '논현역 양식 맛집'}`


[{'url': 'https://www.facebook.com/100064047401838/videos/438709365592330/', 'content': '안녕하세요:) 인천논현역 양식 맛집 심스키친입니다 런치 인기메뉴인 불고기샐러드 피자를 만들었어요ㅎ 상콤한샐러드와 불고기가 치즈와 잘'}, {'url': 'http://m.menupan.com/board/qna_view.asp?bt=r&id=17703', 'content': "논현역 근처에 맛있는 양식집 없나요? ... 논현역 3번출구 '나비' 추천합니다. 나비레스토랑(02-3445-1544) 실은 쌈밥집인데 양식집 찾는데 웬 쌈밥집이냐"}, {'url': 'https://www.diningcode.com/list.dc?query=%EC%8B%A0%EB%85%BC%ED%98%84%20%20%EC%96%91%EC%8B%9D', 'content': '2. 어거스트힐 강남역 ------------- 78점 4.3 (31명) 영업 중 스테이크, 파스타 데이트,무료주차,고급스러운,포장  "가족들과 좋은 일있을때 모시고가기 좋아요. 4. 녘 강남역 --------- 76점 4.4 (10명) 영업 중 퓨전, 파스타 데이트,주차,캐주얼한,콜키지프리  "강남역 뒤 골목에 있는 분위기 좋은 맛집, 파스타 뇨끼 다 맛있어요!" 틴틴 신논현역 ------------ 66점 3.9 (10명) 영업 중 레스토랑, 파스타 데이트,예쁜,반려동물동반  "청첩장 장소로 유명한 곳이라 방문해보았습니다. 바리에가타 신논현점 --------------- 65점 3.6 (7명) 영업 중 뇨끼 데이트,주차,고급스러운  "신논현 근처 파스타 맛집입니다. 연남토마 강남점 ------------- 64점 4.4 (5명) 영업 중 파스타 데이트,주차,깔끔한  "맛있었어요 점심 할인이라 가격도